In [1]:
%pip install -qU langchain-groq

Note: you may need to restart the kernel to use updated packages.


In [11]:
from langchain_community.document_loaders import WebBaseLoader 

loader = WebBaseLoader("https://edition.cnn.com/2024/10/19/politics/us-israel-iran-intelligence-documents/index.html") 

docs = loader.load()


In [12]:
docs

[Document(metadata={'source': 'https://edition.cnn.com/2024/10/19/politics/us-israel-iran-intelligence-documents/index.html', 'title': 'Leaked documents show US intelligence on Israel’s plans to attack Iran, sources say | CNN Politics', 'description': 'The US is investigating a leak of highly classified US intelligence about Israel’s plans for retaliation against Iran, according to three people familiar with the matter. One of the people familiar confirmed the documents’ authenticity.', 'language': 'en'}, page_content=" \n\n\n\n\n\n\n\n\nLeaked documents show US intelligence on Israel’s plans to attack Iran, sources say | CNN Politics\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCNN values your feedback\n\n\n\n\n                                                        1. How relevant is this ad to you?\n                        

In [13]:
from langchain.chains.summarize import load_summarize_chain


In [14]:
import getpass
import os

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")

In [24]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=1,
    max_tokens=None,
    timeout=None,
    # other params...
)



In [27]:
chain = load_summarize_chain(llm, chain_type="refine")
#result = chain.invoke(docs)
#print(result["output_text"])

In [28]:
chain.run(split_docs)

RefineDocumentsChain(verbose=False, initial_llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='Write a concise summary of the following:\n\n\n"{text}"\n\n\nCONCISE SUMMARY:'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7faa12489820>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7faa1248ac90>, model_name='llama-3.1-70b-versatile', temperature=1.0, model_kwargs={}, groq_api_key=SecretStr('**********')), output_parser=StrOutputParser(), llm_kwargs={}), refine_llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['existing_answer', 'text'], input_types={}, partial_variables={}, template="Your job is to produce a final summary.\nWe have provided an existing summary up to a certain point: {existing_answer}\nWe have the opportunity to refine the existing summary (only if needed) with some more context below.\n------------\n{text}\n--

## Map-Reduce Way

## Chuunking document into multiple sub documents

In [29]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=0
)
split_docs = text_splitter.split_documents(docs)
print(f"Generated {len(split_docs)} documents.")

Generated 14 documents.


In [33]:
import operator
from typing import Annotated, List, Literal, TypedDict
from langchain.chains.combine_documents.reduce import (
    acollapse_docs,
    split_list_of_docs,
)
from langchain_core.documents import Document
from langgraph.constants import Send
from langgraph.graph import END, START, StateGraph


In [32]:
%pip install --upgrade --quiet tiktoken langchain langgraph beautifulsoup4

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [21]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

map_prompt = ChatPromptTemplate.from_messages(
    [("system", "Write a concise summary of the following:\\n\\n{context}")]
)

map_chain = map_prompt | llm | StrOutputParser()

In [22]:
reduce_template = """
The following is a set of summaries:
{docs}
Take these and distill it into a final, consolidated summary
of the main themes.
"""

reduce_prompt = ChatPromptTemplate([("human", reduce_template)])

reduce_chain = reduce_prompt | llm | StrOutputParser()



In [23]:
reduce_chain

ChatPromptTemplate(input_variables=['docs'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['docs'], input_types={}, partial_variables={}, template='\nThe following is a set of summaries:\n{docs}\nTake these and distill it into a final, consolidated summary\nof the main themes.\n'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7faa3534d400>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7faa3534e060>, model_name='llama-3.1-70b-versatile', temperature=1.0, model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutputParser()